<div style="text-align: right" align="right"><i>Peter Norvig, 3 Jan 2020</i></div>

# Spelling Bee Puzzle

The [3 Jan. 2020 edition of the **538 Riddler**](https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/) concerns the popular NYTimes  [**Spelling Bee**](https://www.nytimes.com/puzzles/spelling-bee) puzzle:

> In this game, seven letters are arranged in a **honeycomb** lattice, with one letter in the center. Here’s the lattice from Dec. 24, 2019:
> 
> <img src="https://fivethirtyeight.com/wp-content/uploads/2020/01/Screen-Shot-2019-12-24-at-5.46.55-PM.png" width="150">
> 
> The goal is to identify as many words as possible that meet the following criteria:
> 1. The word must be at least four letters long.
> 2. The word must include the central letter.
> 3. The word cannot include any letter beyond the seven given letters.
>
>Note that letters can be repeated. For example,  GAME and AMALGAM are both acceptable words. Four-letter words are worth 1 point each, while five-letter words are worth 5 points, six-letter words are worth 6 points, etc. Words that use all seven letters in the honeycomb are known as **pangrams** and earn 7 bonus points (in addition to the points for the length of the word). So in the above example, MEGAPLEX is worth 8 + 7 = 15 points.
>
> ***Which seven-letter honeycomb results in the highest possible score?*** To be a valid choice of seven letters, no letter can be repeated, it must not contain the letter S (that would be too easy) and there must be at least one pangram.
>
> For consistency, please use [this word list](https://norvig.com/ngrams/enable1.txt) to check your score.


Since the referenced word list came from [***my*** web site](https://norvig.com/ngrams), I felt  compelled to solve this puzzle.  (Note the word list is a standard public domain Scrabble® dictionary that I happen to host a copy of; I didn't curate it, Mendel Cooper and Alan Beale did.) 

I'll show you how I address the problem. First some imports:

In [1]:
from collections import defaultdict
from dataclasses import dataclass
from itertools   import combinations
from typing      import List, Set, Dict, Tuple, Iterable 

# Letters, Words, Lettersets, and Pangrams

Let's start by defining the most basic terms:

- **Valid letter**: the valid letters are uppercase 'A' to 'Z', but not 'S'.
- **Word**: A string of letters.
- **Word list**: a list of valid words.
- **Valid word**: a word of at least 4 valid letters and not more than 7 distinct letters.
- **Letterset**: the set of distinct letters in a word; e.g. letterset('BOOBOO') = 'BO'.
- **Pangram**: a valid word with exactly 7 distinct letters.
- **Pangram letterset**: the letterset for a pangram.

In [2]:
valid_letters = set('ABCDEFGHIJKLMNOPQR' + 'TUVWXYZ')
Letter    = str # A string of one letter
Word      = str # A string of multiple letters
Letterset = str # A sorted string of distinct letters

def word_list(text: str) -> List[Word]: 
    """All the valid words in a text."""
    return [w for w in text.upper().split() if is_valid(w)]

def is_valid(word) -> bool: 
    return len(word) >= 4 and valid_letters.issuperset(word) and len(set(word)) <= 7

def letterset(word) -> Letterset:
    """The set of distinct letters in a word, represented as a sorted str."""
    return ''.join(sorted(set(word)))

def is_pangram(word) -> bool: return len(set(word)) == 7

def pangram_lettersets(wordlist) -> Set[Letterset]:
    """All lettersets from the pangram words in wordlist."""
    return {letterset(w) for w in wordlist if is_pangram(w)}

Why did I  represent a `Letterset` as a sorted string of distinct letters, and not a `set`? Because:
- A `set` can't be the key of a dict.
- A `frozenset` could be a key, and would be a reasonable choice for `Letterset`, but it:
  - Takes up more memory than a `str`.
  - Is harder to read when debugging: `frozenset({'A', 'G', 'L', 'M'})` versus `'AGLM'`.

Here's a mini word list to experiment with:

In [3]:
mini = word_list('amalgam amalgamation cacciatore erotica em game gem gems glam megaplex')
mini

['AMALGAM', 'CACCIATORE', 'EROTICA', 'GAME', 'GLAM', 'MEGAPLEX']

Note that `em` and `gem` are too short, `gems` has an `s`, and `amalgamation` has 8 distinct letters. We're left with six valid words out of the ten candidate words. The pangrams and their lettersets are as follows; there are three pangrams but only two pangram lettersets because CACCIATORE and EROTICA have the same letterset:

In [4]:
assert len(pangram_lettersets(mini)) == 2

{w: letterset(w) for w in mini if is_pangram(w)}

{'CACCIATORE': 'ACEIORT', 'EROTICA': 'ACEIORT', 'MEGAPLEX': 'AEGLMPX'}

# Honeycombs and Scoring

- A **honeycomb** lattice consists of two attributes:
 - A letterset of seven distinct letters
 - A single distinguished center letter
- The **word score** is 1 point for a 4-letter word, or the word length for longer words, plus 7 bonus points for a pangram.
- The **total score** for a honeycomb is the sum of the word scores for the words that the honeycomb **can make**. 
- A honeycomb **can make** a word if the word contains the honeycomb's center, and every letter in the word is in the honeycomb. 

In [5]:
@dataclass(frozen=True, order=True)
class Honeycomb:
    """A Honeycomb lattice, with 7 letters, 1 of which is the center."""
    letters: Letterset 
    center:  Letter
        
def word_score(word) -> int: 
    """The points for this word, including bonus for pangram."""
    return 1 if len(word) == 4 else (len(word) + 7 * is_pangram(word))

def total_score(honeycomb, wordlist) -> int:
    """The total score for this honeycomb."""
    return sum(word_score(w) for w in wordlist if can_make(honeycomb, w))

def can_make(honeycomb, word) -> bool:
    """Can the honeycomb make this word?"""
    return honeycomb.center in word and all(L in honeycomb.letters for L in word)

Here is the honeycomb from the diagram at the top of the notebook:

In [6]:
hc = Honeycomb(letterset('LAPGEMX'), 'G')
hc

Honeycomb(letters='AEGLMPX', center='G')

The word scores, makeable words, and total score  for this honeycomb on the `mini` word list are as follows:

In [7]:
{w: word_score(w) for w in mini}

{'AMALGAM': 7,
 'CACCIATORE': 17,
 'EROTICA': 14,
 'GAME': 1,
 'GLAM': 1,
 'MEGAPLEX': 15}

In [8]:
{w for w in mini if can_make(hc, w)}

{'AMALGAM', 'GAME', 'GLAM', 'MEGAPLEX'}

In [9]:
total_score(hc, mini) # 7 + 1 + 1 + 15

24

# Finding the Top-Scoring Honeycomb

A simple strategy for finding the top (highest total score) honeycomb is:
 - Compile a list of all valid candidate honeycombs.
 - For each honeycomb, compute the total score.
 - Return a (score, honeycomb) tuple for a honeycomb with the maximum score.

In [10]:
def top_honeycomb(wordlist) -> Tuple[int, Honeycomb]: 
    """Find a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    return max((total_score(h, wordlist), h) 
               for h in candidate_honeycombs(wordlist))

What are the possible candidate honeycombs? We could try all letters in all slots, but that's a **lot** of honeycombs:
- The center can be any valid letter (25 choices, because 'S' is not allowed).
- The outside can be any six of the remaining 24 letters.
- All together, that's 25 × (24 choose 6) = 3,364,900 candidate honeycombs.

Fortunately, we can use the constraint that  **there must be at least one pangram** in a valid honeycomb.  So the letters of any valid honeycomb must ***be*** the letterset of some pangram (and the center can be any one of the seven letters):

In [11]:
def candidate_honeycombs(wordlist) -> List[Honeycomb]:
    """Valid honeycombs have pangram letters, with any center."""
    return [Honeycomb(letters, center) 
            for letters in pangram_lettersets(wordlist)
            for center in letters]

In [12]:
candidate_honeycombs(mini) # 7 candidates for each of the 2 pangram lettersets

[Honeycomb(letters='AEGLMPX', center='A'),
 Honeycomb(letters='AEGLMPX', center='E'),
 Honeycomb(letters='AEGLMPX', center='G'),
 Honeycomb(letters='AEGLMPX', center='L'),
 Honeycomb(letters='AEGLMPX', center='M'),
 Honeycomb(letters='AEGLMPX', center='P'),
 Honeycomb(letters='AEGLMPX', center='X'),
 Honeycomb(letters='ACEIORT', center='A'),
 Honeycomb(letters='ACEIORT', center='C'),
 Honeycomb(letters='ACEIORT', center='E'),
 Honeycomb(letters='ACEIORT', center='I'),
 Honeycomb(letters='ACEIORT', center='O'),
 Honeycomb(letters='ACEIORT', center='R'),
 Honeycomb(letters='ACEIORT', center='T')]

Now we're ready to find the highest-scoring honeycomb with respect to the `mini` word list:

In [13]:
top_honeycomb(mini)

(31, Honeycomb(letters='ACEIORT', center='T'))

The program appears to work. But that's just the mini word list. 

# Big Word List

Here's the big word list:

In [14]:
! [ -e  enable1.txt ] || curl -O http://norvig.com/ngrams/enable1.txt
! head  enable1.txt
! wc -w enable1.txt

aa
aah
aahed
aahing
aahs
aal
aalii
aaliis
aals
aardvark
  172820 enable1.txt


In [15]:
file = 'enable1.txt'
big  = word_list(open(file).read())

Here are some statistics:

In [16]:
print(f"""172,820 total words
{len(big):7,d} valid Spelling Bee words
{sum(map(is_pangram, big)):7,d} pangram words
{len(pangram_lettersets(big)):7,d} distinct pangram lettersets
{len(candidate_honeycombs(big)):7,d} candidate honeycombs""")

172,820 total words
 44,585 valid Spelling Bee words
 14,741 pangram words
  7,986 distinct pangram lettersets
 55,902 candidate honeycombs


How long will it take to run `top_honeycomb(big)`? Most of the computation time is in `total_score`, which is called once for each of the 55,902 candidate honeycombs, so let's estimate the total time by first checking how long it takes to compute the total score of a single honeycomb:

In [17]:
%timeit total_score(hc, big)

8.58 ms ± 28.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Roughly 9 milliseconds for one honeycomb. For all 55,902 valid honeycombs:

In [18]:
.009 * 55902

503.11799999999994

About 500 seconds, which is under 9 minutes. I could run `top_honeycomb(big)`, get a coffee, come back, and declare victory. 

But I think that a puzzle like this deserves a more elegant solution.  

# Faster Scoring: Points Table

Here's an idea to make `total_score` faster by doing some precomputation:

1. Do the following computation only once:
  - Compute the `letterset` and `word_score` for each word in the word list. 
  - Make a table of `{letterset: sum_of_word_scores}` giving the total score for each letterset. 
  - I call this a **points table**.
2. For each honeycomb, do the following:
  - Consider every **letter subset** of the honeycomb's 7 letters that includes the center letter.
  - Sum the points table entries for each of these letter subsets.

The resulting algorithm, `fast_total_score`, iterates over just 2<sup>6</sup> – 1 = 63 letter subsets; much fewer than 44,585 valid words. The function `top_honeycomb2` creates the points table and calls `fast_total_score`:

In [19]:
def top_honeycomb2(wordlist) -> Tuple[int, Honeycomb]: 
    """Find a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    table = PointsTable(wordlist)
    return max((fast_total_score(h, table), h) 
               for h in candidate_honeycombs(wordlist))

class PointsTable(dict):
    """A table of {letterset: points} from words."""
    def __init__(self, wordlist):
        for w in wordlist:
            self[letterset(w)] += word_score(w)
    def __missing__(self, key): return 0

def letter_subsets(honeycomb) -> List[Letterset]:
    """The 63 subsets of the letters in the honeycomb, each including the center letter."""
    return [letters 
            for n in range(2, 8) 
            for letters in map(''.join, combinations(honeycomb.letters, n))
            if honeycomb.center in letters]

def fast_total_score(honeycomb, points_table) -> int:
    """The total score for this honeycomb, using a points table."""
    return sum(points_table[s] for s in letter_subsets(honeycomb))

Here is the points table for the mini word list:

In [20]:
table = PointsTable(mini)
table

{'AGLM': 8, 'ACEIORT': 31, 'AEGM': 1, 'AEGLMPX': 15}

The letterset `'AGLM'` gets 8 points (7 for AMALGAM and 1 for GLAM).  `'ACEIORT'` gets 31 points (17 for CACCIATORE and 14 for EROTICA). `'AEGM'` gets 1 for GAME and `'AEGLMPX'` gets 15 for MEGAPLEX. 

Here is the honeycomb `hc` again, and its 63 letter subsets:

In [21]:
hc

Honeycomb(letters='AEGLMPX', center='G')

In [22]:
print(letter_subsets(hc))

['AG', 'EG', 'GL', 'GM', 'GP', 'GX', 'AEG', 'AGL', 'AGM', 'AGP', 'AGX', 'EGL', 'EGM', 'EGP', 'EGX', 'GLM', 'GLP', 'GLX', 'GMP', 'GMX', 'GPX', 'AEGL', 'AEGM', 'AEGP', 'AEGX', 'AGLM', 'AGLP', 'AGLX', 'AGMP', 'AGMX', 'AGPX', 'EGLM', 'EGLP', 'EGLX', 'EGMP', 'EGMX', 'EGPX', 'GLMP', 'GLMX', 'GLPX', 'GMPX', 'AEGLM', 'AEGLP', 'AEGLX', 'AEGMP', 'AEGMX', 'AEGPX', 'AGLMP', 'AGLMX', 'AGLPX', 'AGMPX', 'EGLMP', 'EGLMX', 'EGLPX', 'EGMPX', 'GLMPX', 'AEGLMP', 'AEGLMX', 'AEGLPX', 'AEGMPX', 'AGLMPX', 'EGLMPX', 'AEGLMPX']


The total from `fast_total_score` is the sum of its letter subsets (only 3 of which are in `PointsTable(mini)`):

In [23]:
assert fast_total_score(hc, table) == 24 == table['AGLM'] + table['AEGM'] + table['AEGLMPX']

# Finding the Top-Scoring Honeycomb

We can now solve the puzzle on the big word list:

In [24]:
%time top_honeycomb2(big)

CPU times: user 1.59 s, sys: 3.69 ms, total: 1.59 s
Wall time: 1.59 s


(3898, Honeycomb(letters='AEGINRT', center='R'))

**Wow! 3898 is a high score!** And the whole computation took **less than 2 seconds**!

# Scoring Fewer Honeycombs: Branch and Bound

A run time of less than 2 seconds to find the top honeycomb is pretty good! Can we do even better?

The program would run faster if we scored fewer honeycombs. But if we want to be guaranteed of finding the top-scoring honeycomb, how can we skip any? Consider the pangram **JUKEBOX**. With the unusual letters  **J**, **K**,  and **X**, it scores poorly, regardless of the choice of center:

In [25]:
jk = [Honeycomb(letterset('JUKEBOX'), C) for C in 'JUKEBOX']

{h: total_score(h, big) for h in jk}

{Honeycomb(letters='BEJKOUX', center='J'): 26,
 Honeycomb(letters='BEJKOUX', center='U'): 32,
 Honeycomb(letters='BEJKOUX', center='K'): 26,
 Honeycomb(letters='BEJKOUX', center='E'): 37,
 Honeycomb(letters='BEJKOUX', center='B'): 49,
 Honeycomb(letters='BEJKOUX', center='O'): 39,
 Honeycomb(letters='BEJKOUX', center='X'): 15}

We might be able to dismiss **JUKEBOX** in one call to `fast_total_score`, rather than seven, with this approach:
- Keep track of the top score found so far.
- For each pangram letterset, ask "if we weren't required to use the center letter, what would this letterset score?"
- Check if that score (which is an **upper bound** of the score using any one center letter) is higher than the top score so far.
  - If yes, then try the pangram letterset with all seven centers; 
  - If not then dismiss it without trying any centers.
- This is called a [**branch and bound**](https://en.wikipedia.org/wiki/Branch_and_bound) algorithm: prune a  **branch** of 7 honeycombs if an upper **bound** can't beat the top score.

*Note*: To represent a honeycomb with no center, I can just use `Honeycomb(p, '')`. This works because of a quirk of Python:  `letter_subsets` checks if `honeycomb.center in letters`; normally in Python the expression `e in s` means "*is* `e` *an element of the collection* `s`", but when `s` is a string it means "*is* `e` *a substring of* `s`", and the empty string is a substring of every string. 

I can rewrite `top_honeycomb` as follows:

In [26]:
def top_honeycomb3(words) -> Tuple[int, Honeycomb]: 
    """Find a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    table = PointsTable(words)
    top_score, top_honeycomb = -1, None
    pangrams = [s for s in table if len(s) == 7]
    for p in pangrams:
        if fast_total_score(Honeycomb(p, ''), table) > top_score:
            for center in p:
                honeycomb = Honeycomb(p, center)
                score = fast_total_score(honeycomb, table)
                if score > top_score:
                    top_score, top_honeycomb = score, honeycomb
    return top_score, top_honeycomb

In [27]:
%time top_honeycomb3(big)

CPU times: user 350 ms, sys: 1.78 ms, total: 352 ms
Wall time: 351 ms


(3898, Honeycomb(letters='AEGINRT', center='R'))

Awesome! We get the correct answer, and it runs four times faster than `top_honeycomb2`.

# How many honeycombs does `top_honeycomb3` examine? 

We can use `functools.lru_cache` to make `Honeycomb` keep track:

In [28]:
import functools
Honeycomb = functools.lru_cache(None)(Honeycomb)
top_honeycomb3(big)
Honeycomb.cache_info()

CacheInfo(hits=0, misses=8084, maxsize=None, currsize=8084)

`top_honeycomb3`  examined 8,084 honeycombs; a 6.9× reduction from the 55,902 examined by `top_honeycomb2`. Since there are 7,986 pangram lettersets, that means we had to look at all 7 centers for only (8084-7986)/7 = 14 of them.

How much faster is `fast_total_score` than `total_score` (which takes about 9 milliseconds per honeycomb)?

In [29]:
table = PointsTable(big)

%timeit fast_total_score(hc, table)

26.4 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


We see that `fast_total_score` is about 300 times faster.

# Fancy Report

I'd like to see the actual words that each honeycomb can make, and I'm curious about how the words are divided up by letterset. Here's a function to provide such a report. This report turned out to be more complicated than I anticipated. I guess it is difficult to create a practical extraction and reporting tool. I feel you, [Larry Wall](http://www.wall.org/~larry/).

In [40]:
from textwrap import fill

def report(honeycomb=None, words=big):
    """Print stats, words, and word scores for the given honeycomb (or the top
    honeycomb if no honeycomb is given) over the given word list."""
    bins    = group_by(words, key=letterset)
    if honeycomb is None:
        adj = "Top "
        score, honeycomb = top_honeycomb3(words)
    else:
        adj = ""
        score = total_score(honeycomb, words)
    subsets = letter_subsets(honeycomb)
    nwords  = sum(len(bins[s]) for s in subsets)
    print(f'{adj}{honeycomb}: {Ns(nwords, "word")} {Ns(score, "point")}',
          f'from a {len(words):,d} word list:\n')
    for s in sorted(subsets, key=lambda s: (-len(s), s)):
        if bins[s]:
            pts = sum(word_score(w) for w in bins[s])
            wcount = Ns(len(bins[s]), "pangram" if is_pangram(s) else "word")
            intro = f'{wcount:>8} {Ns(pts, "point"):>9} {{{s}}}: '
            groups= group_by(bins[s], key=word_score)
            strs  = [' '.join(w for w in sorted(groups[s])) + f' ({s})'
                     for s in reversed(sorted(groups))]
            print(fill(intro + ', '.join(strs), width=114, subsequent_indent=' '*4))
            
def Ns(n: int, noun: str) -> str:
    """A string with the int `n` followed by the plural or singular of noun:
    Ns(3, 'bear') => '3 bears'; Ns(1, 'world') => '1 world'"""  
    return f"{n:,d} {noun}{' ' if n == 1 else 's'}"

def group_by(items, key) -> dict:
    "Group items into bins of a dict by key: {key(item): [item, ...], ...}."
    bins = defaultdict(list)
    for item in items:
        bins[key(item)].append(item)
    return bins

Here are reports for the mini and full word lists:

In [41]:
report(hc, mini)

Honeycomb(letters='AEGLMPX', center='G'): 4 words 24 points from a 6 word list:

1 pangram  15 points {AEGLMPX}: MEGAPLEX (15)
 1 word   1 point  {AEGM}: GAME (1)
 2 words  8 points {AGLM}: AMALGAM (7), GLAM (1)


In [42]:
report(words=mini)

Top Honeycomb(letters='ACEIORT', center='A'): 2 words 31 points from a 6 word list:

2 pangrams 31 points {ACEIORT}: CACCIATORE (17), EROTICA (14)


In [43]:
report()

Top Honeycomb(letters='AEGINRT', center='R'): 537 words 3,898 points from a 44,585 word list:

50 pangrams 832 points {AEGINRT}: REAGGREGATING REINTEGRATING (20), ENTERTAINING INTENERATING REGENERATING
    REINITIATING (19), AGGREGATING GRATINEEING INTEGRATING ITINERATING REATTAINING REINTEGRATE REITERATING
    RETARGETING (18), ENTRAINING ENTREATING GARNIERITE GENERATING GREATENING INGRATIATE INTERREGNA INTREATING
    REGRANTING RETRAINING RETREATING (17), ARGENTINE ARGENTITE GARTERING INTEGRATE INTERGANG ITERATING NATTERING
    RATTENING REGRATING RETAGGING RETAINING RETEARING TANGERINE TARGETING TATTERING (16), AERATING GNATTIER
    GRATINEE INTERAGE TREATING (15), GRANITE GRATINE INGRATE TANGIER TEARING (14)
35 words 270 points {AEGINR}: REARRANGING (11), ENGRAINING GANGRENING REENGAGING (10), GARNERING GREGARINE
    REEARNING REGAINING REGEARING (9), AGREEING ANEARING ANGERING ARGININE ENRAGING GRAINIER REGAINER (8), AGINNER
    ANERGIA ANGRIER EARNING EARRING ENGRAIN GEARING GRAI

# 'S' Words

What if we allowed honeycombs to have an 'S' in them? I'll make a new word list that doesn't exclude the 'S'-words, and report on it:

In [39]:
valid_letters.add('S') # Make 'S' a legal letter

big_s = word_list(open(file).read())

report(words=big_s)

Top Honeycomb(letters='AEINRST', center='E'): 1,179 words 8,681 points from a 98,141 word list:

86 pangrams 1,381 points {AEINRST}: ENTERTAINERS INTERSTRAINS STRAITNESSES (19), INTENERATES INTERSTATES
    INTERSTRAIN IRATENESSES ITINERARIES REINITIATES RESTRAINERS TRISTEARINS (18), ANTISTRESS ARTINESSES ENTERTAINS
    ENTRAINERS ENTREATIES ERRANTRIES INTERSTATE INTRASTATE ITINERANTS ITINERATES REINSTATES RESISTANTS RESTRAINER
    RESTRAINTS SANITARIES STANNARIES STRAITNESS TANISTRIES TEARSTAINS TENANTRIES TRANSIENTS TRISTEARIN (17),
    ARSENITES ATTAINERS INSTANTER IRATENESS REATTAINS REINSTATE RESINATES RESISTANT RESTRAINS RESTRAINT RETAINERS
    RETIRANTS SEATRAINS STEARINES STRAINERS STRAITENS TANNERIES TEARSTAIN TERNARIES TRANSIENT (16), ANTISERA
    ARENITES ARSENITE ARTINESS ENTRAINS INERTIAS INTREATS NITRATES RAINIEST RATANIES RESINATE RESTRAIN RETRAINS
    RETSINAS SEATRAIN STAINERS STEARINE STEARINS STRAINER STRAITEN TERRAINS TERTIANS TRAINEES TRAINERS (15),
    ANESTRI ANTS

Allowing 'S' more than doubles the length of the word list and the score of the top honeycomb!

Here are the highest-scoring honeycombs (with and without an S) with their stats and a pangram to remember them by:

<img src="http://norvig.com/honeycombs.png" width="350">
<center>
   537 words &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 1,179 words 
    <br>50 pangrams  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 86 pangrams
    <br>3,898 points &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 8,681 points
    <br> &nbsp;  &nbsp; RETAINING &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; ENTERTAINERS
</center>

# Summary

This notebook showed how to find the highest-scoring honeycomb, starting from a brute-force baseline approach (which we didn't actually run) and modifying the approach with three key improvements:

1. **Brute Force Enumeration**: Compute total score for every honeycomb; return the highest-scoring.
2. **Pangram Lettersets**: Compute total score for  honeycombs that are pangram lettersets (with all 7 centers).
3. **Points Table**: Precompute score for each letterset; for each candidate honeycomb, sum  63 letter subset scores.
4. **Branch and Bound**: Try all 7 centers only for lettersets that score better than the top score so far.

The key ideas paid off in efficiency improvements:


|Approach|Honeycombs|Reduction|`total_score` Time|Speedup|Overall  Time|Overall Speedup|
|--------|----------|--------|----|---|---|---|
|1. **Brute Force Enumeration**|3,364,900|——|9000 microseconds|——|8.5 hours (est.)|——|
|2. **Pangram Lettersets**|55,902|60×|9000 microseconds|——|500 sec (est.)|60×|
|3. **Points Table**|55,902|——|26 microseconds|300×|1.6 seconds|20,000×|
|4. **Branch and Bound**|8,084 |7×|26 microseconds|——|0.35 seconds|90,000×|

